# Orchestration and ML Pipelines

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Ongoing&color=orange)

<!-- Place this tag where you want the button to render. -->
<a class="github-button" href="https://github.com/particle1331/steepest-ascent" data-color-scheme="no-preference: dark; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star particle1331/steepest-ascent on GitHub">Star</a>
<!-- Place this tag in your head or just before your close body tag. -->
<script async defer src="https://buttons.github.io/buttons.js"></script> 


in the previous module we learned about experiment tracking and model registry.
specifically we discussed how to get a candidate model and promote it from staging to production.
in this module, we learn How to automate this process
put in schedule and automate with workflow orchestration specifically prefect. 

workflow orchestration definition 3:40

eliminate negative engineering

prefect

before: DAG

observability 


===

attach observability to each function

model decay over time = drift

    - regularly retrain
    - already have script tools for scheduling
    - want to compare metrics against prod => promote staging to prod

https://thumb.tildacdn.com/tild3462-6534-4732-a462-643534313536/-/format/webp/model_decay_retraini.png


parameter validation prefect



```{margin}
⚠️ **Attribution:** These are notes for [Module 3](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/03-orchestration) of the [MLOps Zoomcamp](https://github.com/DataTalksClub/mlops-zoomcamp). The MLOps Zoomcamp is a free course from [DataTalks.Club](https://github.com/DataTalksClub).
```

## Introduction to Prefect 2.0